In [1]:
rdd=sc.wholeTextFiles("/FileStore/tables/w1j05c2w1508921343218")
f1=rdd.map(lambda kv:(kv[0].split("/")[-1],kv[1]))
f2=f1.map(lambda kv:(kv[0].split(".txt")[0],kv[1]))
f3=f2.flatMapValues(lambda v: v.split("\r\n"))
f4=f3.map(lambda kv:(kv[0],kv[0].split("_")[0],kv[1].split(" ")[0],kv[1].split(" ")[1]))
f5=f4.map(lambda t :(t[1],t[0],t[2],t[3]))

df = sqlContext.createDataFrame(f5)
df=df.toDF('city','shop','month','sales')


In [3]:
#Q1. Average monthly income of the shop in France
row = df.agg({'sales': 'sum'}).collect()
sum_of_sales=row[0]['sum(sales)']
Answer_Q1=sum_of_sales/12
print("Sum of all sales :", sum_of_sales)

print("Average by month :", Answer_Q1)

In [4]:
#Q2. Total revenue per city per year
df.groupby('city').agg({'sales': 'sum'}).orderBy('sum(sales)').show()

In [5]:
#Q3. Average monthly income of the shop in each city
df.groupby('city').agg({'sales': 'mean'}).orderBy('avg(sales)').show()
df.groupby('shop').agg({'sales': 'mean'}).orderBy('avg(sales)').show()


In [7]:
#Q4. Total revenue per store per year
df.groupby('shop').agg({'sales': 'sum'}).orderBy('sum(sales)').show()

In [8]:
#Q5.The store that achieves the best performance in each month
df2=df.groupby("month").agg({'sales': 'max'})
df.join(df2,(df.sales == df2['max(sales)']) & (df.month == df2.month)).select(df.month,df.shop,df2['max(sales)']).show()